In [0]:
%sql
create or replace temp view base_pred as (
    with base as (
        select
        fvend.ID_PORTADOR as ID_PORTADOR
        ,dpdv.UF as UF_PORTADOR
        ,dpdv.GRUPO as GRUPO
        ,sum(fvend.QUANTIDADE) as QTD
        ,count(distinct fvend.ID_VENDA) as QTD_TRANSACOES
        ,max(fvend.QUANTIDADE) as MAIOR_QTD
        ,count(distinct fvend.ID_MARCA) as QTD_MARCAS
    
        from 
            dmn_transformacao_digital_dev.db_analytics.cpv_fvendas as fvend
            left join dmn_transformacao_digital_dev.db_analytics.cpv_dprodutos as dprod on fvend.ID_PRODUTO = dprod.ID_PRODUTO
            left join dmn_transformacao_digital_dev.db_analytics.cpv_dpdv as dpdv on dpdv.ID_LOJA = fvend.ID_LOJA  
            left join dmn_transformacao_digital_dev.db_analytics.cpv_dportador as dport on dport.ID_PORTADOR = fvend.ID_PORTADOR

        where 
            fvend.DATA_COMPRA >= '2025-01-01' and fvend.DATA_COMPRA <= '2025-04-30'
            AND dprod.MARCA = 'ALENIA (A4H)'
        group by all
        ),

        classificacao as (
            select
            fvend.ID_PORTADOR as ID_PORTADOR
            ,dpdv.GRUPO as GRUPO
            
            from 
            dmn_transformacao_digital_dev.db_analytics.cpv_fvendas as fvend
            left join dmn_transformacao_digital_dev.db_analytics.cpv_dprodutos as dprod on fvend.ID_PRODUTO = dprod.ID_PRODUTO
            left join dmn_transformacao_digital_dev.db_analytics.cpv_dpdv as dpdv on dpdv.ID_LOJA = fvend.ID_LOJA  
            left join dmn_transformacao_digital_dev.db_analytics.cpv_dportador as dport on dport.ID_PORTADOR = fvend.ID_PORTADOR

        where 
            fvend.DATA_COMPRA >= '2025-05-01' and fvend.DATA_COMPRA <= '2025-05-31'
            AND dprod.MARCA = 'ALENIA (A4H)'
        group by fvend.ID_PORTADOR, dpdv.GRUPO),

        agrupadores as (
            SELECT
                ID_PORTADOR,
                AVG(dias_desde_anterior) as MED_DIAS_ENTRE_COMPRAS
            FROM (
                SELECT
                    fvend.ID_PORTADOR,
                    fvend.DATA_COMPRA,
                    LAG(fvend.DATA_COMPRA, 1, NULL) OVER (PARTITION BY fvend.ID_PORTADOR ORDER BY fvend.DATA_COMPRA ASC) as DATA_COMPRA_ANTERIOR,
                    DATEDIFF(DATA_COMPRA, DATA_COMPRA_ANTERIOR) as dias_desde_anterior
                FROM
                    dmn_transformacao_digital_dev.db_analytics.cpv_fvendas as fvend
                    left join dmn_transformacao_digital_dev.db_analytics.cpv_dprodutos as dprod on fvend.ID_PRODUTO = dprod.ID_PRODUTO
                    left join dmn_transformacao_digital_dev.db_analytics.cpv_dpdv as dpdv on dpdv.ID_LOJA = fvend.ID_LOJA  
                    left join dmn_transformacao_digital_dev.db_analytics.cpv_dportador as dport on dport.ID_PORTADOR = fvend.ID_PORTADOR
                WHERE
                    fvend.DATA_COMPRA >= '2025-01-01' and fvend.DATA_COMPRA <= '2025-04-30'
                    AND dprod.MARCA = 'ALENIA (A4H)' 
                ) as vendas_com_data_anterior
            WHERE DATA_COMPRA_ANTERIOR IS NOT NULL -- Exclui a primeira compra de cada portador, que não tem anterior
            GROUP BY ID_PORTADOR
            ) 

select 
base.*
,agrupadores.MED_DIAS_ENTRE_COMPRAS
, CASE WHEN isnull(classificacao.ID_PORTADOR) THEN 0 ELSE 1 END as CLASSE

from base as base
left join classificacao on classificacao.id_portador = base.id_portador and base.grupo = classificacao.grupo
left join agrupadores on agrupadores.id_portador = base.id_portador

group by all

        )

In [0]:
%pip install imblearn

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import numpy as np # Usado para np.nan, útil para simulações ou verificações

# Importações do Scikit-learn para pré-processamento e modelo
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer # Importante para aplicar diferentes transformações a diferentes colunas

# Importações do Imbalanced-learn para lidar com desbalanceamento
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline # Pipeline que integra balanceamento

# Importações para avaliação do modelo
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# --- 1. Carregamento dos Dados ---
# Assume que 'spark' (o objeto SparkSession) está disponível no seu ambiente
# e que a view temporária 'base_pred' foi criada com sua query SQL.
print("1. Carregando dados da view 'base_pred' para um DataFrame Pandas...")
pd_base_pred = spark.sql("select * from base_pred").toPandas()
print(f"   Dados carregados. Shape: {pd_base_pred.shape}")


1. Carregando dados da view 'base_pred' para um DataFrame Pandas...
   Dados carregados. Shape: (499351, 9)


In [0]:
# --- 2. Separar Features (X) e Target (y) ---
# Define as colunas que serão usadas como features e a coluna target (CLASSE)
features_cols = [
    'ID_PORTADOR',
    'UF_PORTADOR',
    'GRUPO',
    'QTD',
    'QTD_TRANSACOES',
    'MAIOR_QTD',
    'QTD_MARCAS',
    'MED_DIAS_ENTRE_COMPRAS'
]
target_col = 'CLASSE'

pd_base_pred_x = pd_base_pred[features_cols].copy() # .copy() para evitar SettingWithCopyWarning
pd_base_pred_y = pd_base_pred[target_col].copy()

print("\n2. Colunas de features e target definidas.")
print(f"   Shape de X (features): {pd_base_pred_x.shape}")
print(f"   Shape de y (target): {pd_base_pred_y.shape}")


2. Colunas de features e target definidas.
   Shape de X (features): (499351, 8)
   Shape de y (target): (499351,)


In [0]:
# --- 3. Tratamento de Nulos ---
print("\n3. Verificando e tratando valores nulos...")
print("   Nulos antes do tratamento:")
print(pd_base_pred_x.isnull().sum())

# 3.1. Preencher Nulos em Colunas Numéricas com Zero
# Para features numéricas de compra, NULL geralmente significa ausência de atividade.
colunas_numericas_para_zero = [
    'QTD',
    'QTD_TRANSACOES',
    'MAIOR_QTD',
    'QTD_MARCAS',
    'MED_DIAS_ENTRE_COMPRAS'
]

for col in colunas_numericas_para_zero:
    if col in pd_base_pred_x.columns:
        pd_base_pred_x[col] = pd_base_pred_x[col].fillna(0)

# 3.2. Preencher Nulos em Colunas Categóricas com 'DESCONHECIDO'
# Para colunas categóricas, um nulo deve ser uma categoria própria, não zero.
colunas_categoricas = ['UF_PORTADOR', 'GRUPO']

for col in colunas_categoricas:
    if col in pd_base_pred_x.columns:
        pd_base_pred_x[col] = pd_base_pred_x[col].fillna('DESCONHECIDO')

print("\n   Nulos após o tratamento:")
print(pd_base_pred_x.isnull().sum())


3. Verificando e tratando valores nulos...
   Nulos antes do tratamento:
ID_PORTADOR                    0
UF_PORTADOR                    0
GRUPO                          0
QTD                            0
QTD_TRANSACOES                 0
MAIOR_QTD                      0
QTD_MARCAS                     0
MED_DIAS_ENTRE_COMPRAS    255471
dtype: int64

   Nulos após o tratamento:
ID_PORTADOR               0
UF_PORTADOR               0
GRUPO                     0
QTD                       0
QTD_TRANSACOES            0
MAIOR_QTD                 0
QTD_MARCAS                0
MED_DIAS_ENTRE_COMPRAS    0
dtype: int64


In [0]:
# --- 4. Codificação de Variáveis Categóricas (LabelEncoder) ---
# Aplica LabelEncoder para converter categorias (strings) em números inteiros.
# Isso deve ser feito ANTES do ColumnTransformer, pois ColumnTransformer espera entradas numéricas.
print("\n4. Codificando variáveis categóricas com LabelEncoder...")
for col in colunas_categoricas:
    lencoder = LabelEncoder()
    pd_base_pred_x[col] = lencoder.fit_transform(pd_base_pred_x[col])
    print(f"   Coluna '{col}' codificada para: {pd_base_pred_x[col].unique()}")


4. Codificando variáveis categóricas com LabelEncoder...
   Coluna 'UF_PORTADOR' codificada para: [ 4 25 10 22 17 14 18 15  2 23 16 12  7  9  8 19  5 13  6  1 26 11  3  0
 20 24 21]
   Coluna 'GRUPO' codificada para: [ 4 15  6 24 17 14  9 19 16 11  2 21  3 28  8  1 25 10  7 13  0 22 20  5
 26 23 12 27 18]


In [0]:
# --- 5. Remover ID_PORTADOR das Features ---
# ID_PORTADOR é um identificador e não deve ser usado como feature preditiva.
X_features = pd_base_pred_x.drop(columns=['ID_PORTADOR'])
print(f"\n5. 'ID_PORTADOR' removido das features. Colunas finais para o modelo: {X_features.columns.tolist()}")


5. 'ID_PORTADOR' removido das features. Colunas finais para o modelo: ['UF_PORTADOR', 'GRUPO', 'QTD', 'QTD_TRANSACOES', 'MAIOR_QTD', 'QTD_MARCAS', 'MED_DIAS_ENTRE_COMPRAS']


In [0]:
# --- 6. Divisão dos Dados em Treino e Teste ---
# Divide os dados em conjuntos de treino e teste (80/20).
# 'stratify=pd_base_pred_y' garante que a proporção das classes seja mantida em ambos os conjuntos.
# 'random_state' garante a reprodutibilidade dos resultados.
print("\n6. Dividindo os dados em conjuntos de treino e teste...")
X_train, X_test, y_train, y_test = train_test_split(X_features, pd_base_pred_y,
                                                    test_size=0.2, random_state=42,
                                                    stratify=pd_base_pred_y)

print(f"   Shape dos dados de treino: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"   Shape dos dados de teste: X_test={X_test.shape}, y_test={y_test.shape}")
print(f"   Proporção da Classe 1 no treino: {y_train.value_counts(normalize=True)[1]:.2f}")
print(f"   Proporção da Classe 1 no teste: {y_test.value_counts(normalize=True)[1]:.2f}")


6. Dividindo os dados em conjuntos de treino e teste...
   Shape dos dados de treino: X_train=(399480, 7), y_train=(399480,)
   Shape dos dados de teste: X_test=(99871, 7), y_test=(99871,)
   Proporção da Classe 1 no treino: 0.26
   Proporção da Classe 1 no teste: 0.26


In [0]:
# --- 7. Definição do Pipeline e Parâmetros para GridSearchCV ---

# 7.1. Identifica as colunas para o ColumnTransformer
# Colunas numéricas que precisam ser escaladas (já tratadas para nulos)
numerical_features = [
    'QTD', 'QTD_TRANSACOES', 'MAIOR_QTD', 'QTD_MARCAS', 'MED_DIAS_ENTRE_COMPRAS'
]
# Colunas categóricas que já foram LabelEncoded e devem ser passadas sem escalonamento
categorical_features_encoded = ['UF_PORTADOR', 'GRUPO']

# 7.2. Cria o ColumnTransformer
# 'StandardScaler' será aplicado apenas às 'numerical_features'.
# 'passthrough' será aplicado às 'categorical_features_encoded', ou seja, elas não serão alteradas.
preprocessor = ColumnTransformer(
    transformers=[
        ('num_scaler', StandardScaler(), numerical_features), # Nome 'num_scaler' para o passo de escalonamento numérico
        ('cat_passthrough', 'passthrough', categorical_features_encoded) # Nome 'cat_passthrough' para o passo de categorias
    ])

# 7.3. Define o Pipeline completo
# Inclui o pré-processador, SMOTE e o classificador.
# O 'ImbPipeline' é crucial para que o SMOTE seja aplicado corretamente dentro do K-Fold.
pipeline = ImbPipeline([
    ('preprocessor', preprocessor), # O primeiro passo é o ColumnTransformer para pré-processamento
    ('smote', SMOTE(random_state=42)), # O segundo passo é o balanceamento com SMOTE
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced')) # O último passo é o classificador
])

# 8. Treinamento do Modelo
print("\n8. Iniciando o treinamento do modelo...")
pipeline.fit(X_train, y_train)
print("Treinamento concluído.")





8. Iniciando o treinamento do modelo...
Treinamento concluído.


In [0]:

# 9. Avaliação do Modelo
# Faz previsões no conjunto de teste
y_pred = pipeline.predict(X_test)

# 9.1. Imprimir as métricas de avaliação
print("\nRelatório de Classificação no conjunto de teste:")
print(classification_report(y_test, y_pred))

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

# 9.2. Imprimir a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAcurácia do modelo no conjunto de teste: {accuracy:.4f}")


Relatório de Classificação no conjunto de teste:
              precision    recall  f1-score   support

           0       0.82      0.82      0.82     73507
           1       0.50      0.51      0.50     26364

    accuracy                           0.74     99871
   macro avg       0.66      0.66      0.66     99871
weighted avg       0.74      0.74      0.74     99871


Matriz de Confusão:
[[60030 13477]
 [12982 13382]]

Acurácia do modelo no conjunto de teste: 0.7351
